In [1]:
import numpy as np 
import pandas as pd 
import nltk
from nltk.corpus import stopwords
import string

In [2]:
df = pd.read_csv('emails.txt') #read the CSV file

In [3]:
df.head(5)

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [4]:
df.shape

(5728, 2)

In [5]:
df.columns

Index(['text', 'spam'], dtype='object')

In [7]:
df.drop_duplicates(inplace = True)

In [6]:
df.shape

(5728, 2)

In [7]:
df.isnull().sum()

text    0
spam    0
dtype: int64

In [25]:
from string import punctuation
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import SnowballStemmer

In [26]:
def clean_txt(sent):
    stemmer_s = SnowballStemmer("english")
    tokens = word_tokenize(sent.lower())
    stop_updated = stopwords.words("english") + list(punctuation) + ["..."] + \
    ["would", "could","told","subject:","subject:"]
    text = [stemmer_s.stem(term) for term in tokens if term not in stop_updated and len(term) > 2] 
    #text = [term for term in tokens if term not in stop_updated and len(term) > 2] 
    res = " ".join(text)
    return res

In [27]:
df['text'] = df.text.apply(clean_txt)

In [28]:
df.head()

,text,spam
0,natur irresist corpor ident realli hard recoll...,1
1,stock trade gunsling fanni merril muzo colza a...,1
2,unbeliev new home made easi want show homeown ...,1
3,color print special request addit inform click...,1
4,money get softwar cds softwar compat great gro...,1


In [29]:
X = df.text.values
y = df.spam.values

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=3500)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [32]:
print(X_train_tfidf.shape,X_test_tfidf.shape)

(4009, 3500) (1719, 3500)


In [33]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train_tfidf, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [34]:
y_pred = classifier.predict(X_test_tfidf)

In [35]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [36]:
accuracy_score(y_test,y_pred)

0.9761489237929029

In [37]:
confusion_matrix(y_test,y_pred)

array([[1272,    6],
       [  35,  406]], dtype=int64)